In [1]:
# Make sure we can import EMQST_lib from parent directory
import sys
from pathlib import Path

# Go two levels up from current working directory
PROJECT_ROOT = Path.cwd().resolve().parents[1]   # <- one up (your earlier case)


if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("Using project root:", PROJECT_ROOT)

# Notebook is in project-root/Multiplexed_qubit_readout_results/
# -> project root is the parent of the current working dir
PROJECT_ROOT = Path.cwd().resolve().parent


if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))


import numpy as np 
from datetime import datetime
import os
import uuid
from EMQST_lib import adaptive_functions as ad
from EMQST_lib.qst import QST  
from EMQST_lib import measurement_functions as mf
from EMQST_lib import visualization as vis
from EMQST_lib import support_functions as sf
from EMQST_lib.povm import POVM
%load_ext autoreload
%autoreload 2

Using project root: /workspaces/REMQST


In [ ]:
n_shots = 3000
n_qubits = 2
n_averages = 1
adaptive_burnin = 200

true_states = np.array([sf.generate_random_pure_state(n_qubits) for _ in range(n_averages)])
povm = POVM.generate_Pauli_POVM(n_qubits)
qst = QST(povm, true_states,n_shots,n_qubits,False,{})
print(f'Total shots: {n_shots * n_averages}')
infidelity_container = []
d_st = 0.001
qst_array = []
noise_strengths = []
n_steps = 1
for i in range(n_steps):

    qst.perform_adaptive_BME(depolarizing_strength = i*d_st,
                             adaptive_burnin_steps = adaptive_burnin)
    infidelity_container.append(qst.get_infidelity())
    noise_strengths.append(i * d_st)
    qst_array.append(qst)
    
settings = {
    'n_shots': n_shots,
    'n_qubits': n_qubits,
    'n_averages': n_averages,
    'adaptive_burnin': adaptive_burnin,
    'noise_strengths': noise_strengths,
    'true_states': true_states,
}

path = "adaptive_results/test"
now=datetime.now()
now_string = now.strftime("%Y-%m-%d_%H-%M-%S_")
dir_name= now_string+str(uuid.uuid4())
data_path=f'{path}/{dir_name}'
os.mkdir(data_path)

with open(f'{data_path}/infidelity_container.npy', 'wb') as f:
    np.save(f, infidelity_container)
with open(f'{data_path}/settings.npy', 'wb') as f:    
    np.save(f, settings)


Total shots: 33000
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Completed run 1/1. Final infidelity: 8.682468370047047e-05.
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Completed run 1/1. Final infidelity: 0.00019830414210597613.
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Resampling
Completed run 1/1. Final infidelity: 0.00012525679738784046.
Resampling
Resampling
Resampling
Resampling


KeyboardInterrupt: 

In [ ]:


vis.plot_infidelity_curves(qst)
labels = ['p=0', 'p=0.01', 'p=0.02', 'p=0.03', 'p=0.04']
vis.plot_average_infidelity(infidelity_container, labels = labels)
  


In [ ]:
# Performing non-adaptive and adaptive QST comparison
n_shots = 33000
n_qubits = 1
n_averages = 100
adaptive_burnin = 200

true_states = np.array([sf.generate_random_pure_state(n_qubits) for _ in range(n_averages)])
povm = POVM.generate_Pauli_POVM(n_qubits)
qst_adaptive = QST(povm, true_states,n_shots,n_qubits,False,{})
qst_non_adaptive = QST(povm, true_states,n_shots,n_qubits,False,{})
print(f'Total shots: {n_shots * n_averages}')
noise_strength = 0

qst_adaptive.perform_adaptive_BME(depolarizing_strength =noise_strength,
                            adaptive_burnin_steps = adaptive_burnin)
qst_non_adaptive.perform_adaptive_BME()
infidelity_container=[qst_non_adaptive.get_infidelity(), qst_adaptive.get_infidelity()]
    
settings = {
    'n_shots': n_shots,
    'n_qubits': n_qubits,
    'n_averages': n_averages,
    'adaptive_burnin': [adaptive_burnin, n_shots],
    'noise_strengths': [0,0],
    'true_states': true_states,
}

path = "adaptive_results/adaptive_comparson"
now=datetime.now()
now_string = now.strftime("%Y-%m-%d_%H-%M-%S_")
dir_name= now_string+str(uuid.uuid4())
data_path=f'{path}/{dir_name}'
os.mkdir(data_path)

with open(f'{data_path}/infidelity_container.npy', 'wb') as f:
    np.save(f, infidelity_container)
with open(f'{data_path}/settings.npy', 'wb') as f:    
    np.save(f, settings)

